In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
import tensorflow as tf
import datetime
import os

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
import datetime

# Load dataset
img_size = (64, 64)
batch_size = 32
data_dir = "C:/Users/ovank/dataset"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

# Normalize pixel values
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Improve performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# TensorBoard callback
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 classes: green_light, no_light
])


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



# Train model with TensorBoard logging
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

# Save model
model.save("light_classifier.h5")


Found 170 files belonging to 2 classes.
Using 136 files for training.
Found 170 files belonging to 2 classes.
Using 34 files for validation.
Epoch 1/10


C:\Users\ovank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.6029 - loss: 0.7670 - val_accuracy: 0.6765 - val_loss: 0.6460
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.7132 - loss: 0.6084 - val_accuracy: 0.6765 - val_loss: 0.5829
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.7132 - loss: 0.5220 - val_accuracy: 0.6765 - val_loss: 0.5567
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.7132 - loss: 0.4788 - val_accuracy: 0.7647 - val_loss: 0.4880
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.8750 - loss: 0.4212 - val_accuracy: 0.7941 - val_loss: 0.4613
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.9118 - loss: 0.3824 - val_accuracy: 0.8529 - val_loss: 0.3951
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.9265 - loss: 0.3070 - val_accuracy: 0.9118 - val_loss: 0.2876
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.9265 - loss: 0.2632 - val_accuracy: 0.9118 - val_loss: 0.3242
Epoc

In [9]:
%reload_ext tensorboard
%tensorboard --logdir=fit==logs/fit


Reusing TensorBoard on port 6006 (pid 21540), started 0:00:15 ago. (Use '!kill 21540' to kill it.)

In [7]:
import os
print(os.listdir("logs/fit"))



['20250930-195503']


In [8]:
import os
print(os.listdir("logs/fit/20250930-194306"))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'logs/fit/20250930-194306'

In [ ]:
%tensorboard --logdir=fit==logs/fit


Reusing TensorBoard on port 6006 (pid 15716), started 0:06:11 ago. (Use '!kill 15716' to kill it.)

In [ ]:
import os

log_subdirs = os.listdir(log_dir)
print("Log subdirs:", log_subdirs)

for subdir in log_subdirs:
    files = os.listdir(os.path.join(log_dir, subdir))
    print(f"Files in {subdir}: {files}")


Log subdirs: ['train', 'validation']
Files in train: ['events.out.tfevents.1759254575.ODIN-LAPTOP.20788.8.v2']
Files in validation: ['events.out.tfevents.1759254576.ODIN-LAPTOP.20788.9.v2']


In [ ]:
import tensorflow as tf
import datetime
import os

log_dir = "logs/test/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Dummy dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[..., None]/255.0
x_test = x_test[..., None]/255.0

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28,28,1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 43s 4us/step


C:\Users\ovank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8782 - loss: 0.4669 - val_accuracy: 0.9139 - val_loss: 0.3085
